# 🎮 Sokoban Supervised Learning with PoT

This notebook trains a **Pondering over Thoughts (PoT)** model on Sokoban puzzles using **supervised learning** - identical to our Sudoku training pipeline.

## What is Sokoban?

**Sokoban** (倉庫番, "warehouse keeper") is a classic puzzle game where you push boxes onto target locations.

<img src="https://upload.wikimedia.org/wikipedia/commons/4/4b/Sokoban_ani.gif" width="300" alt="Sokoban gameplay animation">

**Rules:**
- 🧑 Player can move in 4 directions (up/down/left/right)
- 📦 Player can push ONE box at a time (not pull)
- 🎯 Goal: Push ALL boxes onto target squares
- ⚠️ Boxes can get stuck in corners (deadlock = game over!)

▶️ **Watch gameplay:** [Sokoban Tutorial on YouTube](https://www.youtube.com/watch?v=4SjXQ_bHTxU)

**Why is it hard for AI?**
- PSPACE-complete (exponential state space)
- Sparse rewards (only get reward when solved)
- Long-horizon planning required
- Easy to create unsolvable states

## Benchmark Comparison

| Method | Simple (6×6, 1 box) | Complex (10×10, 2 boxes) | Notes |
|--------|---------------------|--------------------------|-------|
| SFT (paper) | ~50% | ~15% | Supervised fine-tuning |
| GPT-4 + LangGraph | varies | varies | [Blog](https://blog.gopenai.com/using-llms-and-langgraph-to-tackle-sokoban-puzzles-5f50b43b9515) |
| RL (PPO) | ~20% | <5% | Very hard to train |
| Random | 25% | 25% | 4 actions |
| **PoT (this notebook)** | TBD | TBD | Adaptive depth |

## Dataset

We use the [Xiaofeng77/sokoban](https://huggingface.co/datasets/Xiaofeng77/sokoban) HuggingFace dataset with:
- ~3,000 (board, optimal_action) pairs
- On-the-fly augmentation (8x via rotations/flips)
- Cross-entropy loss + Q-halt loss (identical to Sudoku)


In [ ]:
# @title 🔧 Setup (Run First)
# @markdown Install dependencies and clone repository

!pip install -q torch datasets tqdm wandb gym-sokoban

# Clone PoT repository
!git clone -q https://github.com/ebenartzy/PoT.git 2>/dev/null || (cd PoT && git pull -q)

import sys
sys.path.insert(0, 'PoT')

print("✅ Setup complete!")


In [ ]:
# @title 📊 Configuration
# @markdown Adjust training parameters

MODEL_TYPE = "pot"  # @param ["pot", "hybrid_pot", "baseline"]
R = 4  # @param {type:"slider", min:1, max:8, step:1}
EPOCHS = 30  # @param {type:"slider", min:5, max:100, step:5}
BATCH_SIZE = 64  # @param {type:"slider", min:16, max:128, step:16}
LEARNING_RATE = 1e-4  # @param {type:"number"}
AUGMENT = True  # @param {type:"boolean"}
USE_WANDB = False  # @param {type:"boolean"}

# Evaluation difficulties
EVAL_DIFFICULTIES = ["simple", "complex"]  # Easy (6x6,1box) and Hard (10x10,2boxes)
EVAL_SAMPLES = 200

print(f"Config: {MODEL_TYPE}, R={R}, epochs={EPOCHS}, batch={BATCH_SIZE}")


In [ ]:
# @title 📥 Load Dataset
# @markdown Downloads Sokoban dataset from HuggingFace

import torch
from torch.utils.data import DataLoader, random_split

from src.data.sokoban_hf import SokobanHFDataset

# Load datasets
print("Loading HuggingFace Sokoban dataset...")
train_ds = SokobanHFDataset(data_dir=".", split="train", augment=AUGMENT)
test_ds = SokobanHFDataset(data_dir=".", split="test", augment=False)

# Split train into train/val
val_size = min(500, len(train_ds) // 5)
train_size = len(train_ds) - val_size
train_subset, val_subset = random_split(
    train_ds, [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

print(f"✅ Train: {len(train_subset)}, Val: {len(val_subset)}, Test: {len(test_ds)}")
print(f"   Board shape: {train_ds.board_shape}")


In [ ]:
# @title 🏗️ Create Model

from src.pot.models.sokoban_solver import SokobanPoT, HybridSokobanPoT, SokobanBaseline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
H, W = train_ds.board_shape

if MODEL_TYPE == "pot":
    model = SokobanPoT(
        board_height=H,
        board_width=W,
        d_model=128,
        n_heads=4,
        n_layers=4,
        R=R,
        num_actions=4,
    )
elif MODEL_TYPE == "hybrid_pot":
    model = HybridSokobanPoT(
        board_height=H,
        board_width=W,
        d_model=128,
        n_heads=4,
        n_layers=4,
        R=R,
        num_actions=4,
    )
else:
    model = SokobanBaseline(
        board_height=H,
        board_width=W,
        d_model=128,
        n_heads=4,
        n_layers=4,
        num_actions=4,
    )

model = model.to(device)
n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"✅ {MODEL_TYPE} model: {n_params:,} parameters")
print(f"   Device: {device}")


In [ ]:
# @title 🚀 Train Model
# @markdown Supervised training with cross-entropy + Q-halt loss (identical to Sudoku)

from src.training.sokoban_supervised import train_supervised

if USE_WANDB:
    import wandb
    wandb.init(project="sokoban-pot", name=f"{MODEL_TYPE}-R{R}")

results = train_supervised(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=1e-4,
    grad_clip=1.0,
    warmup_steps=100,
    use_pot=(MODEL_TYPE != "baseline"),
    wandb_log=USE_WANDB,
)

print(f"\n✅ Training complete!")
print(f"   Best val accuracy: {results['best_val_acc']:.2%}")


In [ ]:
# @title 📈 Evaluate on HuggingFace Test Set

from src.training.sokoban_supervised import evaluate

test_metrics = evaluate(model, test_loader, device)
print(f"\n📊 HuggingFace Test Set Results:")
print(f"   Accuracy: {test_metrics['accuracy']:.2%}")
print(f"   Loss: {test_metrics['loss']:.4f}")


In [ ]:
# @title 🎯 Multi-Difficulty Evaluation
# @markdown Evaluate on Simple (6×6, 1 box) and Complex (10×10, 2 boxes)

from src.data.sokoban_generator import SokobanGeneratedDataset

difficulty_results = {}

for difficulty in EVAL_DIFFICULTIES:
    print(f"\nGenerating {difficulty} test boards...")
    
    eval_ds = SokobanGeneratedDataset(
        difficulty=difficulty,
        n_samples=EVAL_SAMPLES,
        seed=1042,
        augment=False,
    )
    
    eval_loader = DataLoader(eval_ds, batch_size=BATCH_SIZE, shuffle=False)
    
    # Handle different board sizes
    eval_h, eval_w = eval_ds.board_shape
    if (eval_h, eval_w) != train_ds.board_shape:
        print(f"   ⚠️ Board size mismatch: train={train_ds.board_shape}, eval={eval_ds.board_shape}")
        print(f"   Skipping (model trained on fixed size)")
        difficulty_results[difficulty] = {'accuracy': None, 'note': 'size_mismatch'}
        continue
    
    eval_metrics = evaluate(model, eval_loader, device)
    difficulty_results[difficulty] = {
        'accuracy': eval_metrics['accuracy'],
        'loss': eval_metrics['loss'],
    }
    print(f"   {difficulty}: {eval_metrics['accuracy']:.2%} accuracy")

print("\n" + "=" * 50)
print("MULTI-DIFFICULTY RESULTS")
print("=" * 50)
for diff, res in difficulty_results.items():
    if res.get('accuracy') is not None:
        print(f"  {diff}: {res['accuracy']:.2%}")
    else:
        print(f"  {diff}: N/A ({res.get('note', 'error')})")


In [ ]:
# @title 📋 Final Comparison Table

simple_acc = difficulty_results.get('simple', {}).get('accuracy', 0) or 0
complex_acc = difficulty_results.get('complex', {}).get('accuracy', 0) or 0

print("""
╔══════════════════════════════════════════════════════════════════════════╗
║                    SOKOBAN BENCHMARK COMPARISON                          ║
╠══════════════════════════════════════════════════════════════════════════╣
║ Method                   │ Simple (6×6,1) │ Complex (10×10,2) │ Notes    ║
╠══════════════════════════╪════════════════╪═══════════════════╪══════════╣
║ SFT (paper)              │     ~50%       │      ~15%         │ Baseline ║
║ GPT-4 + LangGraph        │     varies     │      varies       │ Zero-shot║
║ RL (PPO)                 │     ~20%       │      <5%          │ Hard     ║
║ Random                   │      25%       │       25%         │ 4 actions║
╠══════════════════════════╪════════════════╪═══════════════════╪══════════╣""")

print(f"║ PoT (this run)           │    {simple_acc:5.1%}      │     {complex_acc:5.1%}        │ R={R}      ║")
print("╚══════════════════════════════════════════════════════════════════════════╝")

# Store for benchmark results
COLAB_RESULTS = {
    'model_type': MODEL_TYPE,
    'R': R,
    'epochs': EPOCHS,
    'best_val_acc': results['best_val_acc'],
    'hf_test_acc': test_metrics['accuracy'],
    'simple_acc': simple_acc,
    'complex_acc': complex_acc,
}
print(f"\n📊 Results dict: {COLAB_RESULTS}")


In [ ]:
# @title 💾 Save Model (Optional)

import os
os.makedirs("checkpoints", exist_ok=True)

save_path = f"checkpoints/sokoban_{MODEL_TYPE}_R{R}.pt"
torch.save({
    'model_state_dict': model.state_dict(),
    'config': {
        'model_type': MODEL_TYPE,
        'R': R,
        'board_shape': train_ds.board_shape,
    },
    'results': COLAB_RESULTS,
}, save_path)

print(f"✅ Model saved to {save_path}")


## 📝 Notes

### Loss Function (Identical to Sudoku)
```python
loss = cross_entropy(action_logits, action_label) + 0.5 * bce(q_halt, is_correct)
```

### Difficulty Levels
| Difficulty | Size | Boxes | Avg Solution Length |
|------------|------|-------|---------------------|
| simple | 6×6 | 1 | ~4 moves |
| larger | 10×10 | 1 | ~15 moves |
| two_boxes | 6×6 | 2 | ~20 moves |
| complex | 10×10 | 2 | ~18 moves |

### References
- [Debunking SFT Generalization](https://arxiv.org/pdf/2510.00237)
- [LLMs + LangGraph for Sokoban](https://blog.gopenai.com/using-llms-and-langgraph-to-tackle-sokoban-puzzles-5f50b43b9515)
- [HuggingFace Dataset](https://huggingface.co/datasets/Xiaofeng77/sokoban)
